In [1]:
import pandas as pd
import numpy as np
import re
from sklearn import preprocessing
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")
pd.set_option("display.max_columns",100)

from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score

In [2]:
def classification_(x,y):
    

    k=KNeighborsClassifier()
    d=DecisionTreeClassifier()
    log=LogisticRegression()
    gbc=GradientBoostingClassifier()
    rf=RandomForestClassifier()
    xgb=XGBClassifier()
    ab=AdaBoostClassifier()

    x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=.2,random_state=42)
    
    algos=[k,d,log,gbc,rf,xgb]
    algos_name=['KNeigbors','DecisionTree','LogisticRegr','GradientBoosting','RandomForest','XGB']
    
    accuracy = []
    precision = []
    recall = []
    f1 = []
   
    result=pd.DataFrame(columns=['AccuracyScore','PrecisionScore','RecallScore','f1_Score'],index=algos_name)
    
    for i in algos:
        
        predict=i.fit(x_train,y_train).predict(x_test)
        
        accuracy.append(accuracy_score(y_test,predict))
        precision.append(precision_score(y_test,predict,average='weighted'))
        recall.append(recall_score(y_test,predict,average='weighted'))
        f1.append(f1_score(y_test,predict,average='weighted'))
        

    
    
    result.AccuracyScore=accuracy
    result.PrecisionScore=precision
    result.RecallScore=recall
    result.f1_Score=f1
    
    
    return result.sort_values('f1_Score',ascending=False)
    
    
    

In [3]:
df1=pd.read_csv('event_type.csv')
df2=pd.read_csv('log_feature.csv')
df3=pd.read_csv('resource_type.csv')
df4=pd.read_csv('severity_type.csv')

In [4]:
df1.head()

,id,event_type
0,6597,event_type 11
1,8011,event_type 15
2,2597,event_type 15
3,5022,event_type 15
4,5022,event_type 11


In [5]:
df2.head()

,id,log_feature,volume
0,6597,feature 68,6
1,8011,feature 68,7
2,2597,feature 68,1
3,5022,feature 172,2
4,5022,feature 56,1


In [6]:
df3.head()

,id,resource_type
0,6597,resource_type 8
1,8011,resource_type 8
2,2597,resource_type 8
3,5022,resource_type 8
4,6852,resource_type 8


In [7]:
df4.head()

,id,severity_type
0,6597,severity_type 2
1,8011,severity_type 2
2,2597,severity_type 2
3,5022,severity_type 1
4,6852,severity_type 1


In [8]:
df5=pd.merge(df1,df3)
df5=pd.merge(df5,df4)
df5=pd.merge(df5,df2)
df5

,id,event_type,resource_type,severity_type,log_feature,volume
0,6597,event_type 11,resource_type 8,severity_type 2,feature 68,6
1,8011,event_type 15,resource_type 8,severity_type 2,feature 68,7
2,2597,event_type 15,resource_type 8,severity_type 2,feature 68,1
3,5022,event_type 15,resource_type 8,severity_type 1,feature 172,2
4,5022,event_type 15,resource_type 8,severity_type 1,feature 56,1
...,...,...,...,...,...,...
146418,8720,event_type 11,resource_type 8,severity_type 1,feature 209,1
146419,6488,event_type 11,resource_type 8,severity_type 2,feature 54,3
146420,878,event_type 11,resource_type 8,severity_type 2,feature 62,1
146421,4464,event_type 11,resource_type 8,severity_type 1,feature 209,1


In [9]:
#for döngüsü kullanma olabildiğince tek satırda hallet(bu önemli)
df5.replace({"event_type ": "", "resource_type ": "", "severity_type ":"", "feature ":""}, regex=True, inplace=True)
df5

,id,event_type,resource_type,severity_type,log_feature,volume
0,6597,11,8,2,68,6
1,8011,15,8,2,68,7
2,2597,15,8,2,68,1
3,5022,15,8,1,172,2
4,5022,15,8,1,56,1
...,...,...,...,...,...,...
146418,8720,11,8,1,209,1
146419,6488,11,8,2,54,3
146420,878,11,8,2,62,1
146421,4464,11,8,1,209,1


In [10]:
df6=pd.read_csv('train.csv')

In [11]:
df6.replace({'location':''},regex=True,inplace=True)
df6

,id,location,fault_severity
0,14121,118,1
1,9320,91,0
2,14394,152,1
3,8218,931,1
4,14804,120,0
...,...,...,...
7376,870,167,0
7377,18068,106,0
7378,14111,1086,2
7379,15189,7,0


In [12]:
y=df6['fault_severity']
#Hedef

In [13]:
df6=df6.drop(['fault_severity'],axis=1)

In [14]:
df7=pd.merge(df6,df5)
df7=df7.drop_duplicates(subset=['id'])#kendini tekrar eden verileri kaldır elinde gerçek veri kalsın
df7.reset_index(drop=True,inplace=True)#index ler karışıktı,yeniden ,indexlendi
df7

,id,location,event_type,resource_type,severity_type,log_feature,volume
0,14121,118,34,2,2,312,19
1,9320,91,34,2,2,315,200
2,14394,152,35,2,2,221,1
3,8218,931,15,8,1,80,9
4,14804,120,34,2,1,134,1
...,...,...,...,...,...,...,...
7376,870,167,34,2,2,232,1
7377,18068,106,20,2,1,219,1
7378,14111,1086,15,8,2,82,21
7379,15189,7,11,8,1,191,3


In [15]:
df7=pd.get_dummies(df7,drop_first=True)
df7

,id,volume,location_ 10,location_ 100,location_ 1000,location_ 1002,location_ 1005,location_ 1006,location_ 1007,location_ 1008,location_ 1009,location_ 101,location_ 1010,location_ 1011,location_ 1013,location_ 1014,location_ 1015,location_ 1016,location_ 1017,location_ 1018,location_ 1019,location_ 102,location_ 1020,location_ 1021,location_ 1022,location_ 1023,location_ 1024,location_ 1025,location_ 1026,location_ 1027,location_ 1029,location_ 103,location_ 1030,location_ 1031,location_ 1032,location_ 1033,location_ 1034,location_ 1035,location_ 1036,location_ 1037,location_ 1038,location_ 104,location_ 1041,location_ 1042,location_ 1044,location_ 1045,location_ 1046,location_ 1047,location_ 1048,location_ 1049,...,log_feature_377,log_feature_378,log_feature_384,log_feature_39,log_feature_40,log_feature_41,log_feature_42,log_feature_44,log_feature_45,log_feature_46,log_feature_47,log_feature_49,log_feature_50,log_feature_51,log_feature_52,log_feature_53,log_feature_54,log_feature_55,log_feature_56,log_feature_60,log_feature_61,log_feature_62,log_feature_63,log_feature_64,log_feature_65,log_feature_66,log_feature_68,log_feature_69,log_feature_70,log_feature_71,log_feature_73,log_feature_74,log_feature_75,log_feature_76,log_feature_77,log_feature_78,log_feature_79,log_feature_8,log_feature_80,log_feature_81,log_feature_82,log_feature_83,log_feature_84,log_feature_85,log_feature_86,log_feature_87,log_feature_90,log_feature_94,log_feature_95,log_feature_99
0,14121,19,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,9320,200,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,14394,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,8218,9,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
4,14804,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7376,870,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
7377,18068,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
7378,14111,21,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
7379,15189,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [16]:
x=pd.DataFrame()
x=df7

In [17]:
classification_(x,y)

[21:53:02] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


,AccuracyScore,PrecisionScore,RecallScore,f1_Score
XGB,0.704807,0.687189,0.704807,0.690954
RandomForest,0.689235,0.665417,0.689235,0.669463
DecisionTree,0.667569,0.668091,0.667569,0.667363
GradientBoosting,0.692620,0.670265,0.692620,0.643046
LogisticRegr,0.651320,0.576799,0.651320,0.573808
KNeigbors,0.589709,0.510613,0.589709,0.528837


In [18]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=.2,random_state=42)

In [19]:
xgb=XGBClassifier()
pred_xgb=xgb.fit(x_train,y_train).predict(x_test)
accuracy_score(y_test,pred_xgb),f1_score(y_test,pred_xgb,average='weighted')

[21:53:19] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


(0.7048070412999323, 0.6909536578595914)

In [20]:
y_pred_proba_xgb=xgb.predict_proba(x_test)

In [21]:
result=pd.DataFrame({'id':x_test.id,
                     'Predicted fault_severity':pred_xgb,
                    'Prediction_probability_0':y_pred_proba_xgb[:,0],
                    'Prediction_probability_1':y_pred_proba_xgb[:,1],
                    'Prediction_probability_2':y_pred_proba_xgb[:,2]},
                   columns=['id','Predicted fault_severity','Prediction_probability_0','Prediction_probability_1','Prediction_probability_2'])

In [22]:
result.head()

,id,Predicted fault_severity,Prediction_probability_0,Prediction_probability_1,Prediction_probability_2
1103,12147,0,0.532838,0.300626,0.166535
2338,2431,0,0.453152,0.451969,0.094879
7201,7821,2,0.173671,0.197937,0.628392
4013,9002,2,0.420823,0.155393,0.423784
2684,11419,0,0.528545,0.331042,0.140413


In [23]:
result.to_csv('service_disruptions.csv')